# Homework 7 Question 4:  Lights Out.
In Tiger Electronic’s handheld solitaire game Lights Out, the player strives to
turn out all 25 lights that make up a 5 × 5 grid of cells. On each turn, the player is allowed to click
on any one cell. Clicking on a cell activates a switch that causes the states of the cell and its (edge)
neighbors to change from on to off, or from off to on. Corner cells are considered to have 2 neighbors,
edge cells to have three, and interior cells to have four. Find a way to turn out all the lights in as few
turns as possible (starting from the state where all lights are on).

## Problem Data

In [1]:
rows = 5
columns = 5;

In [2]:
# Operations allowed to get neighbor of a cell.
# First column element is for row and second column for column operation
ops = [1 0;
       0 1;
      -1 0;
       0 -1]

# Helper function to generate neihbours
function get_nbr(i, j, max_row, max_col)
    retval = []
    for o in 1:length(ops[:,1])
        if i + ops[o,1] <= rows && i + ops[o,1] >= 1
            if j + ops[o,2] <= columns && j + ops[o,2] >= 1
                push!(retval,(i + ops[o,1], j + ops[o,2]))
            end
        end
    end
    return retval
end;

In [3]:
println("Neighbours")
[println("(",i,",",j,"): ",get_nbr(i,j, rows,columns)) for i in 1:rows for j in 1:columns];

Neighbours
(1,1): Any[(2,1),(1,2)]
(1,2): Any[(2,2),(1,3),(1,1)]
(1,3): Any[(2,3),(1,4),(1,2)]
(1,4): Any[(2,4),(1,5),(1,3)]
(1,5): Any[(2,5),(1,4)]
(2,1): Any[(3,1),(2,2),(1,1)]
(2,2): Any[(3,2),(2,3),(1,2),(2,1)]
(2,3): Any[(3,3),(2,4),(1,3),(2,2)]
(2,4): Any[(3,4),(2,5),(1,4),(2,3)]
(2,5): Any[(3,5),(1,5),(2,4)]
(3,1): Any[(4,1),(3,2),(2,1)]
(3,2): Any[(4,2),(3,3),(2,2),(3,1)]
(3,3): Any[(4,3),(3,4),(2,3),(3,2)]
(3,4): Any[(4,4),(3,5),(2,4),(3,3)]
(3,5): Any[(4,5),(2,5),(3,4)]
(4,1): Any[(5,1),(4,2),(3,1)]
(4,2): Any[(5,2),(4,3),(3,2),(4,1)]
(4,3): Any[(5,3),(4,4),(3,3),(4,2)]
(4,4): Any[(5,4),(4,5),(3,4),(4,3)]
(4,5): Any[(5,5),(3,5),(4,4)]
(5,1): Any[(5,2),(4,1)]
(5,2): Any[(5,3),(4,2),(5,1)]
(5,3): Any[(5,4),(4,3),(5,2)]
(5,4): Any[(5,5),(4,4),(5,3)]
(5,5): Any[(4,5),(5,4)]


## Problem Model 

In [4]:
using JuMP, Cbc

m = Model(solver=CbcSolver())

# dummy decision variable to make number of toggles odd
@variable(m, t[1:rows,1:columns] >= 0, Int)
# Number of times a cell has been toggled
@variable(m, toggled[1:rows,1:columns])
# Number of times a cell has been clicked.
@variable(m, clicked[1:rows,1:columns], Bin)

# Constraint for number of toggles to be odd. To switch off the light in the end.
@constraint(m, oddToggles[i=1:rows,j=1:columns], toggled[i,j] == 2*t[i,j] + 1)
# Constraint to make number of toggles equal to the sum of clicks on this cell and its
# neighbours
@constraint(m, nbr[i=1:rows,j=1:columns], clicked[i,j] + sum(clicked[i_n,j_n]
                        for (i_n,j_n) in get_nbr(i,j,rows,columns)) == toggled[i,j])

# Objective to minimize the number of clicks.
@objective(m, Min, sum(clicked[i,j] for i in 1:rows for j in 1:columns))

m

Minimization problem with:
 * 50 linear constraints
 * 75 variables: 25 binary, 25 integer
Solver is CbcMathProg

In [5]:
status=solve(m)
println("Status: ",status)
println("ObjectiveValue: ",getobjectivevalue(m))
clickedCells=getvalue(clicked)
println("Clicks")
[println(clickedCells[i,:]) for i in 1:rows]
numToggles=getvalue(toggled)
println("# of Toggles")
[println(numToggles[i,:]) for i in 1:rows];

Status: Optimal
ObjectiveValue: 15.0
Clicks
[0.0,1.0,1.0,0.0,1.0]
[0.0,1.0,1.0,1.0,0.0]
[0.0,0.0,1.0,1.0,1.0]
[1.0,1.0,0.0,1.0,1.0]
[1.0,1.0,0.0,0.0,0.0]
# of Toggles
[1.0,3.0,3.0,3.0,1.0]
[1.0,3.0,5.0,3.0,3.0]
[1.0,3.0,3.0,5.0,3.0]
[3.0,3.0,3.0,3.0,3.0]
[3.0,3.0,1.0,1.0,1.0]
